In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import numpy as np
import pandas as pd
import time
import sys

In [3]:
import matplotlib.pyplot as plt

In [4]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [5]:
## our hyperparameters optimization for inherently interpretable models
intelligible_bench = pd.read_csv("benchmark_inherently_interpretable.csv",low_memory=False)

## results of hyperparameters optimization for full-complexity models from Grinsztajn et al. (2022)
all_bench = pd.read_csv("benchmark_total.csv",low_memory=False)

### Average number of iteration (for TabSRALinear)

In [6]:
run_df_tabsra = intelligible_bench[(intelligible_bench.model_name=='TabSRA')&(intelligible_bench.model__module__n_head<3) & (intelligible_bench.model__module__n_hidden_encoder==1)]
res = run_df_tabsra.groupby(['benchmark','data__keyword'],as_index=False).agg({'mean_test_score':'count'})
res.mean_test_score.mean()

96.96610169491525

### Get hyperparameter tuning samples from Grinsztajn et al. (2022) for full-complexity models

In [7]:
## This function id used to sample hyperparameter optimization samples form from Grinsztajn et al. (2022)
## The sampling factor is 3 for XGBoost meaning that the number of iterations for XGBoost is 3x the one of TabSRALinear
## in addition we also add the default hyperparameter configuration from Grinsztajn et al. (2022). 
## Therefore we have 3x+1 iteration for XGBoost. 
def sample_from_benchmark(reference_data, bencharmark_data, models=['XGBoost'], sampling_factor = [3],  random_state= 42, metric ='mean_r2_test'):

    tasks = list(reference_data.benchmark.value_counts().index)
    result = []
    #Choice  of the model
    for target_model_id, target_model in enumerate(models):
        bencharmark_data_model = bencharmark_data[(bencharmark_data.model_name==target_model) & (~ bencharmark_data[metric].isnull())]
        target_result = [bencharmark_data_model[bencharmark_data_model.hp=='default']]

        ## choice of the task 
        for id_task, task in enumerate(tasks):
            #target_result = [bencharmark_data_model[(bencharmark_data_model.hp=='default') & ]]
            reference_data_task = reference_data[reference_data.benchmark==task]
            tmp_group = reference_data_task.groupby(['data__keyword']).agg({metric:'count'})
            data_names = tmp_group.index.to_list()
            N_sample_reference = tmp_group.values.flatten().tolist()

            ### choice of the data 
            for data_id, data_name in enumerate(data_names):
                condition = (bencharmark_data_model.hp!='default') & (bencharmark_data_model.data__keyword==data_name)& (bencharmark_data_model.benchmark==task)
                target_result.append(bencharmark_data_model[condition].sample(n= N_sample_reference[data_id]*sampling_factor[target_model_id], random_state = random_state))
        result.append(pd.concat(target_result))
    return pd.concat(result)
        

In [8]:
models = ['XGBoost', 'FT Transformer', 'MLP', 'SAINT', 'Resnet', 'RandomForest']
sampling_factor = [3, 1, 2 ,1 , 1, 2]
all_bench_x = sample_from_benchmark(reference_data=run_df_tabsra, bencharmark_data=all_bench,models=models,sampling_factor = sampling_factor,metric ='mean_test_score')

In [9]:
bench_with_tabr = pd.concat([intelligible_bench,all_bench_x])
bench_with_tabr.shape

(111428, 219)

###  Results

In [10]:
### This function helps to create boostrap like estimate of the test score
### the sampling ratio is 50% and we reapet it for 10 different random seeds
def bootstrap_result(result_data,sampling_ratio=0.5, group_critera = 'model_name',val_metric = 'mean_r2_val', metric ='mean_r2_test', normalize = 'max', clip_0 = True, random_states = [42,43]):
    tasks = list(result_data.benchmark.value_counts().index)
    all_result_all_task = []
    for id_task, task in enumerate(tasks):
        result_data_task = result_data[result_data.benchmark==task]
        tmp_group = result_data_task.groupby(['data__keyword']).agg({metric:'count'})
        data_names = tmp_group.index.to_list()
        all_result = []
        for ind_seed, seed in  enumerate(random_states):   
            result = []
            for data_id, data_name in enumerate(data_names):
                result_target_data_default = result_data_task[(result_data_task.hp=='default') & (result_data_task.data__keyword==data_name)]
                result_target_data_random = result_data_task[(result_data_task.hp!='default') & (result_data_task.data__keyword==data_name)]
                tmp = result_target_data_random.groupby(group_critera).agg({metric:'count'})
                dict_sample_model_per_data = dict(zip(tmp.index.to_list(),(sampling_ratio*tmp.values.flatten()).astype(int).tolist()))
                tmp_sampled = result_target_data_random.groupby(group_critera).apply(lambda group: group.sample(dict_sample_model_per_data[group.name], random_state = seed)).reset_index(drop = True)
                result_target_data_default = result_target_data_default.append(tmp_sampled)
                # sample the best validation performance for each model 
                idx_best_val = result_target_data_default.groupby([group_critera])[val_metric].transform(max) == result_target_data_default[val_metric]
                result_target_data_default = result_target_data_default[idx_best_val].drop_duplicates(subset=group_critera, keep="last")
                result_target_data_default["rank"] = result_target_data_default[metric].rank(ascending=False)
                if clip_0:
                    result_target_data_default[metric] = result_target_data_default[metric].clip(0)
                if normalize=='max':
                    result_target_data_default[metric+'_normalized'] = result_target_data_default[metric]/result_target_data_default[metric].max() 
                if normalize=='minmax':
                    result_target_data_default[metric+'_normalized'] = (result_target_data_default[metric] - result_target_data_default[metric].min())/(result_target_data_default[metric].max() - result_target_data_default[metric].min())
                result_target_data_default["bootstrap_seed"] = seed
                
                result.append(result_target_data_default)
            all_result.append(pd.concat(result))
        all_result_all_task.append(pd.concat(all_result))
    return pd.concat(all_result_all_task)

### Comparison of models

In [11]:
## For default TabSRALinear is ( N_hidden_encoder =1) and N_head in {1,2}
## we optimize this for 9 days/216 hours
## TabSRA refers to the TabSRALinear model 
cond_1 = (bench_with_tabr.model_name=='TabSRA')&(bench_with_tabr.model__module__n_hidden_encoder==1)&(bench_with_tabr.model__module__n_head<3)
cond = cond_1|(bench_with_tabr.model_name!='TabSRA')
inter_model = bench_with_tabr[cond]

boost_x_normalize_max = bootstrap_result(result_data = inter_model, random_states = [42,43,44,45,46,47,48,49,50, 51],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')
boost_x_normalize_max_all = bootstrap_result(result_data = inter_model, sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')

In [12]:
boost_x_normalize_max.shape, boost_x_normalize_max_all.shape

((6490, 222), (649, 222))

#### Bootstrap like results (for 10 different random seeds)

In [13]:
res = boost_x_normalize_max.groupby(["model_name"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_test_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median"]})
res.iloc[[0,1,2,4,9,5,7,8,3,6,10]].style.background_gradient(cmap='coolwarm')

#### Results for all iterations

In [14]:
res = boost_x_normalize_max_all.groupby(["model_name"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_test_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median"]})
res.iloc[[0,1,2,4,9,5,7,8,3,6,10]].style.background_gradient(cmap='coolwarm')

#### Result on all iterations
- x $\approx$ 95 per dataset for TabSRALinear
- we consider 3x for XGBoost, 2x for RF, MLP and 1x for FT-Transformer, SAINT, ResNet
- For all inherenlty interpretable models, Decision Tree (DT) , Linear/Logistic Regression (LR), Explainable Boosting Machine (EBM)
and EBM without interactions terms (EBM_S), we use the same budget of optimization: 9days/216hours on 64-core processor CPU. 

## Ablation study (for TabSRALinear)

In [15]:
def sample_data(base_data, group_critera = 'model__module__dim_head', metric='mean_test_score',random_state=42):

    tasks = list(base_data.benchmark.value_counts().index)
    result = []
    ## choice of the task 
    for id_task, task in enumerate(tasks):
        target_result = []
        reference_data_task = base_data[base_data.benchmark==task]
        tmp_group = reference_data_task.groupby(['data__keyword']).agg({metric:'count'})
        data_names = tmp_group.index.to_list()
        #N_sample_reference = tmp_group.values.flatten().tolist()

        ### choice of the data 
        for data_id, data_name in enumerate(data_names):
            condition = reference_data_task.data__keyword==data_name
            N_sample = reference_data_task[condition].groupby(group_critera).agg({metric:'count'})[metric].min()
            target_result.append(reference_data_task[condition].groupby(group_critera).sample(n= N_sample, random_state = random_state))
        result.append(pd.concat(target_result))
    return pd.concat(result)
        

### Study influence of dimension $d_k$ 
we study the influence of $d_k$  when the number of ensembles/heads is one 

In [16]:
inter_dim_attention = inter_model[inter_model.model__module__n_head==1]
inter_dim_attention =  sample_data(inter_dim_attention)

In [17]:
tmp = inter_dim_attention.groupby(['data__keyword','model__module__dim_head'],as_index=False).agg({'mean_val_score':'count'}).rename(columns={'mean_val_score':'Nb'})
tmp.Nb.mean(),tmp.Nb.median(),tmp.Nb.max()

(17.666666666666668, 16.0, 36)

**For each for each configuration of dataset, n head==1 and $d_k \in \{4,8,12\}$, we run $\approx 17$ hyperparameters optimizations iterations**

In [18]:
boost_x_normalize_max_all = bootstrap_result(result_data = inter_dim_attention,group_critera = 'model__module__dim_head', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')
boost_x_normalize_max_all_train = bootstrap_result(result_data = inter_dim_attention,group_critera = 'model__module__dim_head', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_train_score', metric ='mean_train_score')

#### test score (corresponding to the best validation validation configuration)

In [19]:
res = boost_x_normalize_max_all.groupby(["model__module__dim_head"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_test_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median"]})
res.style.background_gradient(cmap='coolwarm')

#### best training configuration

In [20]:
boost_x_normalize_max_all_train.groupby(["model__module__dim_head"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_train_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median", "count"]}) \
.style.background_gradient(cmap='coolwarm')

### Study influence of H: number of  ensembles/heads 
we study the influence of $H$  when the number of the encoder's hidden layer is set one 

In [21]:
cond_2 = (bench_with_tabr.model_name=='TabSRA')&(bench_with_tabr.model__module__n_hidden_encoder==1)
n_head_attention = bench_with_tabr[cond_2]
n_head_attention =  sample_data(n_head_attention,group_critera='model__module__n_head')

In [22]:
tmp = n_head_attention.groupby(['data__keyword','model__module__n_head'],as_index=False).agg({'mean_val_score':'count'}).rename(columns={'mean_val_score':'Nb'})
tmp.Nb.min(),tmp.Nb.mean(),tmp.Nb.median(),tmp.Nb.max()

(10, 29.866666666666667, 26.0, 65)

**For each for each configuration of dataset, n hidden_encoder==1, $H  \in \{1,2,3,4,5,6\}$, we run $\approx 30$ hyperparameters optimizations iterations**

In [23]:
boost_x_normalize_max_all = bootstrap_result(result_data = n_head_attention,group_critera = 'model__module__n_head', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')
boost_x_normalize_max_all_train = bootstrap_result(result_data = n_head_attention,group_critera = 'model__module__n_head', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_train_score', metric ='mean_train_score')

#### test score (corresponding to the best validation validation configuration)

In [24]:
res = boost_x_normalize_max_all.groupby(["model__module__n_head"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_test_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median"]})
res.style.background_gradient(cmap='coolwarm')

#### best training configuration

In [26]:
boost_x_normalize_max_all_train.groupby(["model__module__n_head"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_train_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median", "count"]}) \
.style.background_gradient(cmap='coolwarm')

### Study influence of n_hidden_encoder: the number of hidden layer in the Query/Key encoder 
we study the influence of n_hidden_encoder for n_head $\in \{1,2\}$ and $d_k \in \{4,8,12\}$

In [27]:
cond_3 = (bench_with_tabr.model_name=='TabSRA')&(bench_with_tabr.model__module__n_hidden_encoder==1)&(bench_with_tabr.model__module__n_head<3)
cond = cond_3|(bench_with_tabr.model__module__n_hidden_encoder!=1)&(bench_with_tabr.model__module__n_head<3)
n_hidden_encoder_attention = bench_with_tabr[cond]
n_hidden_encoder_attention['model__module__n_hidden_encoder'] = n_hidden_encoder_attention['model__module__n_hidden_encoder'].fillna(2)

In [28]:
tmp = n_hidden_encoder_attention.groupby(['data__keyword','model__module__n_hidden_encoder'],as_index=False).agg({'mean_val_score':'count'}).rename(columns={'mean_val_score':'Nb'})
tmp.Nb.mean(),tmp.Nb.median(),tmp.Nb.max()

(115.57777777777778, 97.0, 242)

**For each for each configuration of dataset, n head $\in \{1,2\}$ and $d_k \in \{4,8,12\}$, we run $\approx 115$ hyperparameters optimizations iterations** in order to study the influence of the n_hidden_encoder 

In [29]:
n_hidden_encoder_attention =  sample_data(n_hidden_encoder_attention,group_critera='model__module__n_hidden_encoder')

In [30]:
tmp = n_head_attention.groupby(['data__keyword','model__module__n_hidden_encoder'],as_index=False).agg({'mean_val_score':'count'}).rename(columns={'mean_val_score':'Nb'})
tmp.Nb.min(),tmp.Nb.mean(),tmp.Nb.median(),tmp.Nb.max()

(60, 179.2, 156.0, 390)

In [31]:
boost_x_normalize_max_all = bootstrap_result(result_data = n_hidden_encoder_attention,group_critera = 'model__module__n_hidden_encoder', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')
boost_x_normalize_max_all_train = bootstrap_result(result_data = n_hidden_encoder_attention,group_critera = 'model__module__n_hidden_encoder', sampling_ratio=1.0,random_states = [42],normalize='max', val_metric = 'mean_train_score', metric ='mean_train_score')

In [32]:
boost_x_normalize_max_all.groupby(["model__module__n_hidden_encoder"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_test_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median", "count"]}) \
.style.background_gradient(cmap='coolwarm')

In [33]:
boost_x_normalize_max_all_train.groupby(["model__module__n_hidden_encoder"],as_index=False).agg({'rank':["min","max","mean","median"],'mean_train_score'+'_normalized':["mean","median", "std"],'mean_time':["mean", "median", "count"]}) \
.style.background_gradient(cmap='coolwarm')

In [34]:
res = boost_x_normalize_max_all.groupby(["model__module__n_hidden_encoder","rank"]).agg({'data__keyword':["count"]})
res.style.background_gradient(cmap='coolwarm')

In [38]:
#using all iterations instead of the bootstrap (with 10 different random seeds)
#boost_x_normalize_max_all = bootstrap_result(result_data = inter_model, sampling_ratio=1.0,random_states = [42], val_metric = 'mean_val_score', metric ='mean_test_score')

In [49]:
boost_x_normalize_max = bootstrap_result(result_data = inter_model, random_states = [42,43,44,45,46,47,48,49,50, 51],normalize='max', val_metric = 'mean_val_score', metric ='mean_test_score')


In [50]:
boost_x_normalize_max.shape

(6490, 222)

In [51]:
res = boost_x_normalize_max.groupby(['benchmark','data__keyword','model_name'],as_index=False).agg({'mean_test_score':["mean","std"],'mean_time':"mean"}).round(3)
#res = res.fillna(0)

In [52]:
res

benchmark                        data__keyword  \
                                                                              
0    categorical_classification_medium                               albert   
1    categorical_classification_medium                               albert   
2    categorical_classification_medium                               albert   
3    categorical_classification_medium                               albert   
4    categorical_classification_medium                               albert   
5    categorical_classification_medium                               albert   
6    categorical_classification_medium                               albert   
7    categorical_classification_medium                               albert   
8    categorical_classification_medium                               albert   
9    categorical_classification_medium                               albert   
10   categorical_classification_medium                               albert   
11   categorical_classification_medium                     compas-two-years   
12   categorical_classification_medium                     compas-two-years   
13   categorical_classification_medium                     compas-two-years   
14   categorical_classification_medium                     compas-two-years   
15   categorical_classification_medium                     compas-two-years   
16   categorical_classification_medium                     compas-two-years   
17   categorical_classification_medium                     compas-two-years   
18   categorical_classification_medium                     compas-two-years   
19   categorical_classification_medium                     compas-two-years   
20   categorical_classification_medium                     compas-two-years   
21   categorical_classification_medium                     compas-two-years   
22   categorical_classification_medium                            covertype   
23   categorical_classification_medium                            covertype   
24   categorical_classification_medium                            covertype   
25   categorical_classification_medium                            covertype   
26   categorical_classification_medium                            covertype   
27   categorical_classification_medium                            covertype   
28   categorical_classification_medium                            covertype   
29   categorical_classification_medium                            covertype   
30   categorical_classification_medium                            covertype   
31   categorical_classification_medium                            covertype   
32   categorical_classification_medium                            covertype   
33   categorical_classification_medium       default-of-credit-card-clients   
34   categorical_classification_medium       default-of-credit-card-clients   
35   categorical_classification_medium       default-of-credit-card-clients   
36   categorical_classification_medium       default-of-credit-card-clients   
37   categorical_classification_medium       default-of-credit-card-clients   
38   categorical_classification_medium       default-of-credit-card-clients   
39   categorical_classification_medium       default-of-credit-card-clients   
40   categorical_classification_medium       default-of-credit-card-clients   
41   categorical_classification_medium       default-of-credit-card-clients   
42   categorical_classification_medium       default-of-credit-card-clients   
43   categorical_classification_medium       default-of-credit-card-clients   
44   categorical_classification_medium                          electricity   
45   categorical_classification_medium                          electricity   
46   categorical_classification_medium                          electricity   
47   categorical_classification_medium                          electricity   
48   categorical_classification_medium                          electricity   


### Example of Difficult tasks for TabSRALinear (and MLP like architecture)

In [53]:
features = ['model_name','mean_test_score','std_test_score','mean_time']

In [57]:
res[(res.benchmark=='numerical_regression_medium')&(res.data__keyword=='delays_zurich_transport')].iloc[[0,1,2,4,9,5,7,8,3,6,10]]

benchmark            data__keyword      model_name  \
                                                                            
506  numerical_regression_medium  delays_zurich_transport    DecisionTree   
507  numerical_regression_medium  delays_zurich_transport             EBM   
508  numerical_regression_medium  delays_zurich_transport           EBM_S   
510  numerical_regression_medium  delays_zurich_transport          Linear   
515  numerical_regression_medium  delays_zurich_transport          TabSRA   
511  numerical_regression_medium  delays_zurich_transport             MLP   
513  numerical_regression_medium  delays_zurich_transport          Resnet   
514  numerical_regression_medium  delays_zurich_transport           SAINT   
509  numerical_regression_medium  delays_zurich_transport  FT Transformer   
512  numerical_regression_medium  delays_zurich_transport    RandomForest   
516  numerical_regression_medium  delays_zurich_transport         XGBoost   

    mean_test_score        mean_time  
               mean    std      mean  
506           0.018  0.000     0.017  
507           0.027  0.000    17.792  
508           0.026  0.000     4.725  
510           0.005  0.000     9.960  
515           0.012  0.001    47.965  
511           0.013  0.001    22.449  
513           0.011  0.001    50.765  
514           0.021  0.001    58.392  
509           0.019  0.001    51.534  
512           0.031  0.002     5.921  
516           0.030  0.001     8.577

In [47]:
res[(res.benchmark=='numerical_regression_medium')&(res.data__keyword=='yprop_4_1')].iloc[[0,1,2,4,9,5,7,8,3,6,10]]

benchmark data__keyword      model_name  \
                                                                 
638  numerical_regression_medium     yprop_4_1    DecisionTree   
639  numerical_regression_medium     yprop_4_1             EBM   
640  numerical_regression_medium     yprop_4_1           EBM_S   
642  numerical_regression_medium     yprop_4_1          Linear   
647  numerical_regression_medium     yprop_4_1          TabSRA   
643  numerical_regression_medium     yprop_4_1             MLP   
645  numerical_regression_medium     yprop_4_1          Resnet   
646  numerical_regression_medium     yprop_4_1           SAINT   
641  numerical_regression_medium     yprop_4_1  FT Transformer   
644  numerical_regression_medium     yprop_4_1    RandomForest   
648  numerical_regression_medium     yprop_4_1         XGBoost   

    mean_test_score        mean_time  
               mean    std      mean  
638           0.026  0.007     0.006  
639           0.056  0.002     4.138  
640           0.048  0.001     1.686  
642           0.043  0.003    19.606  
647           0.023  0.020    26.594  
643           0.014  0.010     9.927  
645           0.013  0.017    28.791  
646           0.057  0.005   165.168  
641           0.045  0.005   162.342  
644           0.092  0.002     5.152  
648           0.080  0.004    11.171